In [259]:
# import modules
import numpy as np

# import plotting modules
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

from IPython.display import Latex

%matplotlib inline

from matplotlib import gridspec
from NFW_Z import NFW_Z

In [260]:
class AvDensProf:
    
    
    def __init__(self, Rv):
        
        """Initiate the class with a known virial radius of the halo.
        Input: virial radius in kpc."""
        
        self.Rvir = Rv


    def find_dens_profile(self, raw_radius_data, radius_array, bin_size):
        
        """Derive an AVERAGE dark matter density profile from raw data.
        Inputs:
        1) raw_radius_data: array of the radii at which dark matter particles are found, based on provided data.
        2) radius_array: array of the radii of imaginary spherical shells centered on the galactic center.
        3) bin_size: integer that best matches the image resolution.
        Output: density_profile (the AVERAGE dark matter density profile of the data)"""
        
        density_profile = np.zeros(np.size(radius_array))
        i = 0
        for radius_value in radius_array:
            particles = np.where((raw_radius_data < (radius_value + bin_size)))
            how_many_particles = np.size(particles)
            shell_volume = 4/3*np.pi*((radius_value + bin_size)**3)
            density_profile[i] = how_many_particles / shell_volume * 500 #500 is 500 M_Sun, the mass per particle
            i += 1
        return density_profile
    
    
    def extension(self, starting_radius, upper_limit_guess, bin_size):
        
        """If the SUBFIND virial radius isn't correct, the researcher can use this one to create
        an extended radius array that covers the region where they think the actual virial radius
        should be.
        Inputs:
        1) starting_radius: where to begin the array
        2) upper_limit_guess: where to end the array
        3) bin_size: how big each increment is between two elements of the array.
        Output: extended_array (the extended radius array)"""
        
        extended_array = np.arange(starting_radius, upper_limit_guess, bin_size)
        return extended_array
    
    
    def deriving_r200(self, rho_crit, upper_limit_guess, bin_size, numerator, addition):
        
        """Using the extended radius array from extension, this function generates an extended density profile
        that's shaped like the graph of 1/x^3, then finds the radius on the extended radius array that best
        matches a density equal to 200 times the critical density on the density array. 
        Inputs:
        1) rho_crit: the critical density in this specific case.
        2) upper_limit_guess: where to end the array
        3) bin_size: how big each increment is between two elements of the array.
        4) numerator: the numerator that would give the best extended density profile plot
        5) addition: the addition to the denominator that would give the best extended density profile plot
        Output: rad200 (the radius where the density best matches 200 times rho_crit)"""
        
        # extract the extended radius array from extension
        extended_array = self.extension(starting_radius, upper_limit_guess, bin_size)
        
        # use the extended radius array to make an extended density profile
        extended_dens_prof = numerator / (extended_array**3 + addition)        
        deviation = np.abs(extended_dens_prof - rho_crit*200)
        least_difference_index = deviation.argmin()
        rad200 = extended_array[least_difference_index]
        return rad200
        
        
    def deriving_r_RDO(self, rho_crit, rho_delta_omega, upper_limit_guess, bin_size, numerator, addition):

        """Using the extended radius array from extension, this function generates an extended density profile
        that's shaped like the graph of 1/x^3, then finds the radius on the extended radius array that best
        matches a density equal to rho_crit * delta_vir * Omega_M. 
        Inputs:
        1) rho_crit: the critical density in this specific case.
        2) rho_delta_omega: rho_crit * delta_vir * Omega_M. This value is dependent on the redshift observed.
        3) upper_limit_guess: where to end the array
        4) bin_size: how big each increment is between two elements of the array.
        5) numerator: the numerator that would give the best extended density profile plot
        6) addition: the addition to the denominator that would give the best extended density profile plot
        Output: rad_rdo (the radius where the density best matches rho_crit * delta_vir * Omega_M)"""
        
        extended_array = self.extension(starting_radius, upper_limit_guess, bin_size)
        extended_dens_prof = numerator / (extended_array**3 + addition)        
        difference = abs(extended_dens_prof - rho_delta_omega)
        smallest_diff_index = difference.argmin()
        rad_rdo = extended_array[smallest_diff_index]
        return rad_rdo
        
        
    def final_plot(self, starting_radius, bin_size, raw_radius_data, radius_array, upper_limit_guess, \
                   quoted_Rvir, rho_crit, rho_delta, rho_delta_omega, numerator, addition):
        
        """This function generates the final plot, with the AVERAGE density profile based on provided data,
        the EXTENDED density profile, as well as two pairs of similarly colored horizontal and vertical lines.
        Inputs:
        1) starting_radius: where to begin the radius array. Unit: kpc
        2) bin_size: how big each increment is between two elements of the radius array. Unit: kpc
        3) raw_radius_data: array of the radii at which dark matter particles are found, based on provided data.
        4) radius_array: array of the radii of imaginary spherical shells centered on the galactic center.
        5) upper_limit_guess: where to end the radius array. Unit: kpc
        6) quoted_Rvir: the virial radius quoted in the data. Unit: kpc
        7) rho_crit: the critical density in this case.
        8) rho_delta = rho_crit * delta_vir (delta_vir depends on the observed redshift)
        9) rho_delta_omega = rho_crit * delta_vir * Omega_M (delta_vir and Omega_M depend on the observed redshift)
        10) numerator: the numerator that would give the best extended density profile plot
        11) addition: the addition to the denominator that would give the best extended density profile plot
        Output: the final plot."""
        
        # extract the density profile array from find_dens_profile
        density_profile = self.find_dens_profile(raw_radius_data, radius_array, bin_size)
        
        # extract the extended radius array from extension and make an extended density profile based on it
        extended_array = self.extension(starting_radius, upper_limit_guess, bin_size)
        extended_dens_prof = numerator / (extended_array**3 + addition)
        
        # extract rad200 from deriving_r200
        rad200 = self.deriving_r200(rho_crit, upper_limit_guess, bin_size, numerator, addition)

        # extract rad_rdo from deriving r_RDO
        rad_rdo = self.deriving_r_RDO(rho_crit, rho_delta_omega, upper_limit_guess, bin_size, numerator, addition)
        
        # we all know what this is
        fig, ax = plt.subplots(figsize=[10, 10])
        
        # plot the AVERAGE density profile based on data. 
        # this line will be purple.
        plt.plot(radius_array, density_profile, color=(0.5, 0.1, 0.9, 1), linewidth=2, \
                 label="average density profile based on data")
        
        # plot the radius (and corresponding density) where the density best matches rho_crit * delta_vir.
        # these lines will be cyan.
        plt.axhline(rho_delta, color='#23B5D3', linewidth=2, \
                   label="HORIZONTAL: $ρ_{crit}$ x $Δ_{vir}$" + \
                    "\nVERTICAL: quoted $r_{vir}$: " + str(round(quoted_Rvir, 2)) + ' kpc')
        plt.axvline(quoted_Rvir, color='#23B5D3', linewidth=2)

        # plot the extended density profile
        plt.plot(extended_array, extended_dens_prof, color='#2B4570', linestyle='-.')

        # plot the radius (and corresponding density) where the density best matches rho_crit * 200
        # these lines will be red.
        plt.axhline(rho_crit * 200, color='#EA526F', linewidth=2, \
                   label="HORIZONTAL: $ρ_{crit}$ x 200" + "\nVERTICAL: $r_{200}$ = " + str(round(rad200, 2)) + " kpc")
        plt.axvline(rad200, color='#EA526F', linewidth=2)

        # plot the radius (and corresponding density) where the density best matches rho_crit * delta_vir * Omega_M.
        # these lines will be grey.
        plt.axhline(rho_delta_omega, color='#D0CFCF', linewidth=2, \
                   label="HORIZONTAL: $ρ_{crit}$ x $Δ_{vir}$ x $\Omega_{M}$" + \
                   '\nVERTICAL: actual $r_{vir}$ needed: ' + str(rad_rdo) + ' kpc')
        plt.axvline(rad_rdo, color='#D0CFCF', linewidth=2)

        # plot the points of intersection
        plt.plot(quoted_Rvir, rho_delta, 'o', color='#23B5D3')
        plt.plot(rad200, rho_crit * 200, 'o', color='#EA526F')
        plt.plot(rad_rdo, rho_delta_omega, 'o', color='#D0CFCF')

        # this deals with the legends and the rest
        plt.legend(bbox_to_anchor=(1.02, 1.01), loc='upper left')
        plt.xlabel('distance from galactic center (kpc)', fontsize=17)
        plt.ylabel('average dark matter density ($M_\odot$/kpc$^3$)', fontsize=17) # using LATEX here!
        plt.title('Plot Title', loc='center', fontsize=17)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.xscale('log')
        plt.yscale('log')    

        # lo and behold!
        plt.show()